# Full Archive Search

- Coppying form this github page https://github.com/twitterdev/Twitter-API-v2-sample-code/blob/master/Full-Archive-Search/full-archive-search.py

In [86]:
import time
import pandas as pd

# to sleep between calls

In [109]:
import requests
import os
import json

# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = os.environ.get("BEARER_TOKEN")

#bearer_token = ''

search_url = "https://api.twitter.com/2/tweets/search/all"

# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
query_params = { #"query":"from:CNNChile lang:es",
                'query': '(from:saund97 -is:retweet) ',
                'tweet.fields': 'author_id,id,created_at',
                "start_time":"2019-10-01T07:20:50.52Z",
                "end_time":"2019-11-01T07:20:50.52Z",
                "max_results": "10"
}

#query_params = {'query': 'from:' + user + ' -fromDate:' + startDate + '-toDate:' + endDate}

# jacs query # 'query': 'from:' + user + ' -fromDate:' + startDate + '-toDate:' + endDate


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers, params):
    response = requests.request("GET", search_url, headers=headers, params=params)
    print(response.status_code)
    if response.status_code != 200:
        print(response.status_code)
        raise Exception(response.status_code, response.text)
    return response.json()


def main():
    headers = create_headers(bearer_token)
    json_response = connect_to_endpoint(search_url, headers, query_params)
    results = json.dumps(json_response, indent=4, sort_keys=True)
    #print(json.dumps(json_response, indent=4, sort_keys=True))

    return results


In [100]:
results = main()

200


In [101]:
y = json.loads(results)

In [103]:
y

{'meta': {'result_count': 0}}

In [75]:
len(y['data'])

10

In [76]:
y['meta']['next_token']

'b26v89c19zqg8o3fnm91i40gu1gp7vh7j0r6ddrjyhvul'

In [78]:
z = json.loads(next_results)

In [79]:
len(z['data'])

10

In [81]:
z['data'][:2]

[{'author_id': '19017675',
  'created_at': '2019-10-27T10:22:53.000Z',
  'id': '1188400668983414786',
  'text': "Are Remainer parties holding the country 'hostage over' Brexit?\n#FarageOnLBC https://t.co/mmYNZ5dPdX"},
 {'author_id': '19017675',
  'created_at': '2019-10-26T22:29:02.000Z',
  'id': '1188221022601990145',
  'text': 'Phenomenal performance by my friend Derek tonight. https://t.co/k40HLKUGDa'}]

In [134]:

# Get all results from one user

num_runs = 0
all_results = []

result_count = 100  # initilise this as a non-zero number
                    # will be filled with respnses from json later 
                    # can get a max of 100 records at a time
                    # so if the results arnt 100 we ran out of things to get

while result_count == 100:

    time.sleep(1)

    if num_runs ==0:
        # No next_token parameter
        query_params = { #"query":"from:CNNChile lang:es",
                'query': '(from:Nigel_Farage -is:retweet) ',
                'tweet.fields': 'author_id,id,created_at',
                "start_time":"2019-10-01T07:20:50.52Z",
                "end_time":"2019-11-01T07:20:50.52Z",
                "max_results": "100"
        }
    else:
        # with next_token
        query_params = { #"query":"from:CNNChile lang:es",
                'query': '(from:Nigel_Farage -is:retweet) ',
                'tweet.fields': 'author_id,id,created_at',
                "start_time":"2019-10-01T07:20:50.52Z",
                "end_time":"2019-11-01T07:20:50.52Z",
                "max_results": "100",
                "next_token": results_json['meta']['next_token'] # get token from before: 'b26v89c19zqg8o3fnm91i40gu1gp7vh7j0r6ddrjyhvul'
        }

    headers = create_headers(bearer_token)
    json_response = connect_to_endpoint(search_url, headers, query_params)
    results = json.dumps(json_response, indent=4, sort_keys=True)

    results_json = json.loads(results)

    # append this pagnation to all results
    #all_results.append(results_json)
    result_count = results_json['meta']['result_count']
    num_runs = num_runs + 1
    print(num_runs, result_count)

    # Only add results if theyre non zero
    if result_count > 0:
        list_of_tweet_dicts = results_json['data']

        user = 'Nigel_Farage'
        for tweet in list_of_tweet_dicts:
            all_results.append([user, tweet['created_at'], tweet['id']])

    




print('finished')

200
1 83
finished


In [136]:
all_results[:10]

[['Nigel_Farage', '2019-10-31T19:37:00.000Z', '1189989669620080641'],
 ['Nigel_Farage', '2019-10-31T18:07:33.000Z', '1189967159176388611'],
 ['Nigel_Farage', '2019-10-31T16:58:43.000Z', '1189949832821137408'],
 ['Nigel_Farage', '2019-10-31T11:08:41.000Z', '1189861745009053697'],
 ['Nigel_Farage', '2019-10-29T20:44:06.000Z', '1189281779603124227'],
 ['Nigel_Farage', '2019-10-29T18:34:42.000Z', '1189249212447215616'],
 ['Nigel_Farage', '2019-10-29T18:06:55.000Z', '1189242221121495041'],
 ['Nigel_Farage', '2019-10-28T19:18:00.000Z', '1188897721085771777'],
 ['Nigel_Farage', '2019-10-27T18:58:28.000Z', '1188530419635052551'],
 ['Nigel_Farage', '2019-10-27T17:23:11.000Z', '1188506439742119939']]

# Turn into function for a given user

Got it to work here, try to functionise it

In [155]:

# Get all results from one user

# TODO: Documentation
def users_tweets(user, start_time, end_time):
# Given a user return a list of all of there tweets on october 2019
# start_time and end_time are not yet implemented

    num_runs = 0
    all_results = []

    result_count = 100  # initilise this as a non-zero number
                        # will be filled with respnses from json later 
                        # can get a max of 100 records at a time
                        # so if the results arnt 100 we ran out of things to get

    while result_count == 100:

        time.sleep(1)

        if num_runs ==0:
            # No next_token parameter
            query_params = { #"query":"from:CNNChile lang:es",
                    'query': f'(from:{user} -is:retweet) ',
                    'tweet.fields': 'author_id,id,created_at',
                    "start_time":"2019-10-01T07:20:50.52Z",
                    "end_time":"2019-11-01T07:20:50.52Z",
                    "max_results": "100"
            }
        else:
            # with next_token
            query_params = { #"query":"from:CNNChile lang:es",
                    'query': f'(from:{user} -is:retweet) ',
                    'tweet.fields': 'author_id,id,created_at',
                    "start_time":"2019-10-01T07:20:50.52Z",
                    "end_time":"2019-11-01T07:20:50.52Z",
                    "max_results": "100",
                    "next_token": results_json['meta']['next_token'] # get token from before: 'b26v89c19zqg8o3fnm91i40gu1gp7vh7j0r6ddrjyhvul'
            }

        headers = create_headers(bearer_token)
        json_response = connect_to_endpoint(search_url, headers, query_params)
        results = json.dumps(json_response, indent=4, sort_keys=True)

        results_json = json.loads(results)

        # append this pagnation to all results
        #all_results.append(results_json)
        result_count = results_json['meta']['result_count']
        num_runs = num_runs + 1
        #print(num_runs, result_count)

        # Only add results if theyre non zero
        if result_count > 0:
            list_of_tweet_dicts = results_json['data']

            for tweet in list_of_tweet_dicts:
                all_results.append([user, tweet['created_at'], tweet['id']])

    return all_results

In [156]:
users_tweets('nigel_farage',0,0)

200


[['nigel_farage', '2019-10-31T19:37:00.000Z', '1189989669620080641'],
 ['nigel_farage', '2019-10-31T18:07:33.000Z', '1189967159176388611'],
 ['nigel_farage', '2019-10-31T16:58:43.000Z', '1189949832821137408'],
 ['nigel_farage', '2019-10-31T11:08:41.000Z', '1189861745009053697'],
 ['nigel_farage', '2019-10-29T20:44:06.000Z', '1189281779603124227'],
 ['nigel_farage', '2019-10-29T18:34:42.000Z', '1189249212447215616'],
 ['nigel_farage', '2019-10-29T18:06:55.000Z', '1189242221121495041'],
 ['nigel_farage', '2019-10-28T19:18:00.000Z', '1188897721085771777'],
 ['nigel_farage', '2019-10-27T18:58:28.000Z', '1188530419635052551'],
 ['nigel_farage', '2019-10-27T17:23:11.000Z', '1188506439742119939'],
 ['nigel_farage', '2019-10-27T10:22:53.000Z', '1188400668983414786'],
 ['nigel_farage', '2019-10-26T22:29:02.000Z', '1188221022601990145'],
 ['nigel_farage', '2019-10-26T14:46:07.000Z', '1188104526911496192'],
 ['nigel_farage', '2019-10-26T11:55:50.000Z', '1188061673992667137'],
 ['nigel_farage', '2

In [142]:
CNNChile_tweets = users_tweets('CNNChile',0,0)

200
1 100
200
2 100
200
3 100
200
4 100
200
5 100
200
6 100
200
7 100
200
8 100
200
9 100
200
10 100
200
11 100
200
12 100
200
13 100
200
14 100
200
15 100
200
16 100
200
17 100
200
18 100
200
19 100
200
20 100
200
21 100
200
22 29
finished


In [143]:
len(CNNChile_tweets)

2129

In [153]:
CNNChile_tweets[:5]

[['CNNChile', '2019-11-01T01:51:01.000Z', '1190083790993051649'],
 ['CNNChile', '2019-10-31T23:38:54.000Z', '1190050543005241344'],
 ['CNNChile', '2019-10-31T22:38:55.000Z', '1190035450943168512'],
 ['CNNChile', '2019-10-31T21:39:43.000Z', '1190020552402059267'],
 ['CNNChile', '2019-10-31T21:13:35.000Z', '1190013975624335368']]

In [151]:
# Convert them to pandas
df_cnn = pd.DataFrame.from_records(CNNChile_tweets,columns=['User', 'TweetCreated', 'TweetId'])
df_cnn.head()

,User,TweetCreated,TweetId
0,CNNChile,2019-11-01T01:51:01.000Z,1190083790993051649
1,CNNChile,2019-10-31T23:38:54.000Z,1190050543005241344
2,CNNChile,2019-10-31T22:38:55.000Z,1190035450943168512
3,CNNChile,2019-10-31T21:39:43.000Z,1190020552402059267
4,CNNChile,2019-10-31T21:13:35.000Z,1190013975624335368


In [149]:
df_cnn.to_csv('CNNChile_tweets_Oct.csv') # Save

In [ ]:
# TODO: create funtion to go through every news source we have

In [159]:

# Function to get all tweet ID's from a given date range for a list of users

def get_list_users_tweets(users, start_date, end_date):
    '''
    users - a list of twitter user
    first_id - start date + time we want #TODO: Implement
    last_id - end date + time we want   #TODO: Implement
    '''

    res = []

    for user in users:
        
        print('-'*30)
        print(f"User: {user} Number: {users.index(user)+1} of {len(users)}")
        print('-'*30)
        print()
        # Append results of function to results
        # Extend will insure everying is in the same list
        # Append would create a list of lists
        res.extend( users_tweets(user, start_date, end_date ) )

    #Convert results from list of list to DataFrame
    tweets_df = pd.DataFrame.from_records(res, columns=['User', 'TweetCreated', 'TweetId'])

    return tweets_df


# Chilian News sources

### So far these are the sources that have been identified as potentially interesting:

- El Mercurio (@ElMercurio_cl) / Twitter
- La Tercera (@latercera) / Twitter
- laSegunda (@La_Segunda) / Twitter
- Las Últimas Noticias (@lun) / Twitter
- La Cuarta (@lacuarta) / Twitter
- Meganoticias (@meganoticiascl) / Twitter
- Canal 13 (@canal13) / Twitter
- TVN (@TVN) / Twitter
- 24 Horas (@24HorasTVN) / Twitter
- La Nación Chile (@nacioncl) / Twitter
- Diario Financiero (@DFinanciero) / Twitter
- Chilevisión (@chilevision) / Twitter
- UCV Radio 103.5 FM (@ucvradio) / Twitter
- LaRed (@LaRedTV) / Twitter
- Cooperativa (@Cooperativa) / Twitter
- Radio Pudahuel (@RadioPudahuel) / Twitter
- BioBioChile (@biobio) / Twitter
- El Conquistador 91.3 (@FMConquistador) / Twitter
- EL INFORMADORCHILE🇨🇱 (@INFORMADORCHILE) / Twitter
- CNN Chile (@CNNChile) / Twitter
- CHV Noticias (@CHVNoticias) / Twitter
- El Mostrador (@elmostrador) / Twitter
- Publimetro (@PublimetroChile) / Twitter
- El Desconcierto (@eldesconcierto) / Twitter
- El Dínamo (@el_dinamo) / Twitter
- El Ciudadano (@El_Ciudadano) / Twitter
- El Líbero (@elliberocl) / Twitter
- (4) Diario El Observador (@eo_enlinea) / Twitter




In terms of the time period, if you could look between October 18, 2019 and November 18, 2019, that would be some cool gravy.




In [161]:
chile_news_users = ['ElMercurio_cl',
                    'latercera',
                    'La_Segunda',
                    'lun',
                    'lacuarta',
                    'meganoticiascl',
                    'canal13',
                    'TVN',
                    '24HorasTVN',
                    'nacioncl',
                    'DFinanciero',
                    'chilevision',
                    'ucvradio',
                    'LaRedTV',
                    'Cooperativa',
                    'RadioPudahuel',
                    'biobio',
                    'FMConquistador',
                    'INFORMADORCHILE',
                    'CNNChile',
                    'CHVNoticias',
                    'elmostrador',
                    'PublimetroChile',
                    'eldesconcierto',
                    'el_dinamo',
                    'El_Ciudadano',
                    'elliberocl',
                    'eo_enlinea']

# TODO: turn into actual dates
start_date = 1185187691857678337  # 2019-10-17
end_date = 1195811463706873857    # 2019-11-16
now_id = 1395883494652383232  # 2021-05-21

# Try getting just the dates we want in 2019
results = get_list_users_tweets(chile_news_users[:5], start_date, end_date)


------------------------------
User: ElMercurio_cl Number: 1 of 5
------------------------------

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
------------------------------
User: latercera Number: 2 of 5
------------------------------

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
------------------------------
User: La_Segunda Number: 3 of 5
------------------------------

200
------------------------------
User: lun Number: 4 of 5
------------------------------

200
200
200
200
200
200
------------------------------
User: lacuarta Number: 5 of 5
------------------------------

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [162]:
results

,User,TweetCreated,TweetId
0,ElMercurio_cl,2019-11-01T02:50:00.000Z,1190098635121283072
1,ElMercurio_cl,2019-11-01T02:40:00.000Z,1190096118329135104
2,ElMercurio_cl,2019-11-01T02:30:00.000Z,1190093602464129024
3,ElMercurio_cl,2019-11-01T02:20:00.000Z,1190091085407825921
4,ElMercurio_cl,2019-11-01T02:10:00.000Z,1190088568707923969
...,...,...,...
9598,lacuarta,2019-10-01T11:00:00.000Z,1178987924056150016
9599,lacuarta,2019-10-01T10:45:00.000Z,1178984148930826240
9600,lacuarta,2019-10-01T10:41:29.000Z,1178983264146657283
9601,lacuarta,2019-10-01T10:30:00.000Z,1178980373931479040


In [163]:
results.to_csv('first_5_news_sources.csv')

In [ ]:
results